In [99]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
#nltk.download('punkt') # one time execution
import re
import unidecode
from tqdm import tqdm

In [100]:
df=pd.read_excel('Prueba de relatos.xlsx')
df.head()

,Capitulo,Texto
0,La Placita de la Recoleta \n,LA PLACITA de la RECOLETA\nA Francisco Mendoza...
1,La Plazuela de la Salud \n,LA PLAZUELA de la SALUD\nA Baltazar Caravedo y...
2,La Plazuela de San Carlos\n,LA PLAZUELA de SAN CARLOS\nSALIENDO DEL ANTIGU...
3,La Plazuela de Santa Catalina\n,LA PLAZUELA de SANTA CATALINA\nA Alfonso Espin...
4,La Plazuela de San Pedro \n,LA PLAZUELA de SAN PEDRO\nCURIOSA CALLE es en ...


In [102]:
sentences = []
#for s in df['Texto']:
s=df['Texto'][0]
sentences.append(sent_tokenize(s))

In [103]:
# flatten the list
#sentences = [y for x in sentences for y in x]
sentences = [unidecode.unidecode(y) for x in sentences for y in x]
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [104]:
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')

In [105]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [106]:
cl=[]
for i in clean_sentences:
    p=i.split(' ')
    p=' '.join(p)
    p=re.sub(r'\n','',p)
    p=p.strip()
    cl.append(p)

In [107]:
word_embeddings = {}
f = open('SBW-vectors-300-min5.txt', encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

1000654it [01:41, 9839.13it/s] 


In [108]:
sentence_vectors = []
for i in cl:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((300,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((300,))
    sentence_vectors.append(v)

In [109]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [110]:

from sklearn.metrics.pairwise import cosine_similarity

In [111]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,300), sentence_vectors[j].reshape(1,300))[0,0]

In [112]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [114]:

# Specify number of sentences to form the summary
sn = 5
# Generate summary
for i in range(sn):
      print(ranked_sentences[i][1])

LA PLACITA de la RECOLETA
A Francisco Mendoza y Canaval
SALIENDO DE LA CALLE DE BELEN, la mejor y mas aristocratica de las calles de la capital, residencia de las familias de buen tono: Soyer y Cabero, Alvarez Calderon, CanavalSoria, Alayza, Irribarren, etc., etc., entre las cuales se ha venido a colar, no se sabe como, una fabrica de cigarrillos, la del gordo judio aleman Cohen, propietario de las afamadas marcas "Garibaldi", "Almirante Grau" y "Huascar", y "como quien se va" a la exposicion, se llega a un trozo de pena colocado en la esquina a manera de guardacanton y sobre el que todos los chicos del barrio cada vez que pasan junto a el, tienen la ineludible obligacion de dar un airoso "salto peruano con patada" o sin ella.
Un pasadizo a la izquierda lleva a la capilla espaciosa con palanganas media naranja y capulino y con recuerdos delicados que alguna limena antigua relata al instante; "Don Bartolome, el fundador, quiso que la capilla se levantase en el mismo sitio en que sepulta